## Домашнее задание 2: Линейные модели. Работа с признаками

Правила:

* Домашнее задание оценивается в 10 баллов.

* Можно использовать без доказательства любые результаты, встречавшиеся на лекциях или семинарах по курсу, если получение этих результатов не является вопросом задания.

* Можно использовать любые свободные источники с *обязательным* указанием ссылки на них.

* Плагиат не допускается.

* Старайтесь сделать код как можно более оптимальным и читаемым.

In [306]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%pylab inline

Populating the interactive namespace from numpy and matplotlib


В этом задании мы рассмотрим различные аспекты построения линейной модели. Мы будем работать с одним из классических наборов данных в статистике, содержащим информацию о бриллиантах. Описание можно посмотреть [здесь](https://www.kaggle.com/shivam2503/diamonds).

In [307]:
data = pd.read_csv('diamonds.csv')
data.head(5)

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


Мы будем решать задачу предсказания цены бриллианта `price` в зависимости от его характеристик.

**Задание 1 (1 балл)** Есть ли в наборе данных пропущенные значения? Если да, удалите их. 

In [308]:
#проверим
data.isnull().sum()

Unnamed: 0    0
carat         0
cut           0
color         0
clarity       0
depth         0
table         0
price         0
x             0
y             0
z             0
dtype: int64

In [309]:
# -> пропущенных значений нет

**Задача 2 (1 балл)** Есть ли в наборе данных бессмысленные столбцы (признаки, не несущие дополнительной информации)? Если да, то удалите их. Поясните свой выбор (напишите текстом обоснование).

In [310]:
#первый столбец это просто индекс -> он бесполезен
data.drop('Unnamed: 0',axis=1,inplace=True)

**Задание 3 (1 балл)** Линейная регрессия основана на предположении о линейной связи между признаками и целевой переменной, а потому перед выбором переменных для включения в модель имеет смысл проверить, насколько эта связь выполняется. Для следующих пунктов нам также потребуются корреляции между признаками. Выведите матрицу корреляций между всеми вещественными признаками и целевой переменной.

Какие вещественные признаки коррелируют с целевой переменной больше всего?

In [311]:
#Выведите матрицу корреляций между всеми вещественными признаками и целевой переменной
data.corr()['price']

carat    0.921591
depth   -0.010647
table    0.127134
price    1.000000
x        0.884435
y        0.865421
z        0.861249
Name: price, dtype: float64

In [312]:
data.corr()['price'].sort_values(ascending=False)

#Какие вещественные признаки коррелируют с целевой переменной больше всего?
#Ответ - масса/длина/ширина/глубина коррелируют больше всего из вещественных признаков

price    1.000000
carat    0.921591
x        0.884435
y        0.865421
z        0.861249
table    0.127134
depth   -0.010647
Name: price, dtype: float64

**Задание 4 (1 балл)** Так как линейная модель складывает значения признаков с некоторыми весами, нам нужно аккуратно обработать категориальные признаки. Закодируйте категориальные переменные при помощи OneHot-кодирования (pd.get_dummies). Не забудьте поставить значение параметра drop_first равным True.

In [313]:
# вещественные признаки 
# пригодится для StandartScaler()
numerical_culumns = data.select_dtypes(include='number').columns.drop('price')
numerical_culumns

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [314]:
#аккуратно кодирую
data = pd.get_dummies(data,drop_first=True)
data

,carat,depth,table,price,x,y,z,cut_Good,cut_Ideal,cut_Premium,...,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,0.23,61.5,55.0,326,3.95,3.98,2.43,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1,0.21,59.8,61.0,326,3.89,3.84,2.31,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2,0.23,56.9,65.0,327,4.05,4.07,2.31,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0.29,62.4,58.0,334,4.20,4.23,2.63,0,0,1,...,0,1,0,0,0,0,0,1,0,0
4,0.31,63.3,58.0,335,4.34,4.35,2.75,1,0,0,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53935,0.72,60.8,57.0,2757,5.75,5.76,3.50,0,1,0,...,0,0,0,0,1,0,0,0,0,0
53936,0.72,63.1,55.0,2757,5.69,5.75,3.61,1,0,0,...,0,0,0,0,1,0,0,0,0,0
53937,0.70,62.8,60.0,2757,5.66,5.68,3.56,0,0,0,...,0,0,0,0,1,0,0,0,0,0
53938,0.86,61.0,58.0,2757,6.15,6.12,3.74,0,0,1,...,1,0,0,0,0,1,0,0,0,0


**Задание 5 (1 балл)** 
Создайте матрицу X, содержащую все признаки, и не содержащую целевую переменную price. Также создайте вектор y, содержащий целевую переменную price.

In [315]:
X = data.drop('price',axis=1)
X

,carat,depth,table,x,y,z,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,...,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,0.23,61.5,55.0,3.95,3.98,2.43,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0.21,59.8,61.0,3.89,3.84,2.31,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,0.23,56.9,65.0,4.05,4.07,2.31,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0.29,62.4,58.0,4.20,4.23,2.63,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
4,0.31,63.3,58.0,4.34,4.35,2.75,1,0,0,0,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53935,0.72,60.8,57.0,5.75,5.76,3.50,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
53936,0.72,63.1,55.0,5.69,5.75,3.61,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
53937,0.70,62.8,60.0,5.66,5.68,3.56,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
53938,0.86,61.0,58.0,6.15,6.12,3.74,0,0,1,0,...,1,0,0,0,0,1,0,0,0,0


In [316]:
y = data['price']
y

0         326
1         326
2         327
3         334
4         335
         ... 
53935    2757
53936    2757
53937    2757
53938    2757
53939    2757
Name: price, Length: 53940, dtype: int64

**Задание 5 (1 балл)** 
Перемешайте данные! 

Разделите выборку на тренировочную и тестовую. Долю тестовой выборки укажите равной 0.3.

In [317]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=39)

**Задание 6 (1 балл)** Зачастую при использовании линейных моделей вещественные признаки масштабируются.  В этой задаче масштабируйте вещественные признаки тренировочной и тестовой выборок при помощи модуля `StandardScaler`.

После применения масштабирования матрица перестает быть объектом Pandas Dataframe - решите эту проблему.

In [318]:
from sklearn.preprocessing import StandardScaler
# создаём масштабировщик
scaler = StandardScaler()
# обучаем его на тренировочных данных
scaler.fit(Xtrain[numerical_culumns])
# трансформируем тестовые и тренировочные данные
# и
# одновременно решаем проблему с конвертацией типов: numpy.ndarray -> pd.DataFrame
Xtrain[numerical_culumns] = scaler.transform(Xtrain[numerical_culumns])
Xtest[numerical_culumns] = scaler.transform(Xtest[numerical_culumns])

**Задание 7 (1 балл)** Обучите линейную регрессию на тренировочной выборке. Выведите r2-score на тренировочной и тестовой выборках.

In [319]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

model = LinearRegression()
model.fit(Xtrain, y=ytrain)

LinearRegression()

In [320]:
ytest_predict = model.predict(Xtest)
ytrain_predict = model.predict(Xtrain)

In [321]:
print(f'r2-score на тренировочной выборке: {round(r2_score(ytrain, ytrain_predict),3)}')
print(f'r2-score на тестовой выборке: {round(r2_score(ytest, ytest_predict),3)}')

r2-score на тренировочной выборке: 0.922
r2-score на тестовой выборке: 0.914


**Задание 8 (1 балл)** Выведите на экран веса, которые линейная регрессия присвоила признакам. Назовите вещественные переменные, оценки коэффициентов которых по модулю на порядок превышают оценки прочих вещественных переменных.

In [322]:
#веса
weights = model.coef_
#признаки
features = model.feature_names_in_
# датафрейм с признаками и весами
features_and_weights = pd.DataFrame({'features':features, 'weights':weights})

In [323]:
#сортировка признаков по убыванию модуля оценок их весов
features_and_weights.sort_values('weights',key=abs,ascending=False)

,features,weights
0,carat,5395.008793
16,clarity_IF,4992.162155
21,clarity_VVS1,4679.583846
22,clarity_VVS2,4643.640684
19,clarity_VS1,4273.470379
20,clarity_VS2,3961.828656
17,clarity_SI1,3353.043331
18,clarity_SI2,2371.602090
15,color_J,-2368.369214
3,x,-1514.732540


In [324]:
numerical_culumns

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [325]:
# Назовите вещественные переменные, оценки коэффициентов которых по модулю на порядок превышают оценки прочих вещественных переменных ->

# -> Если имеются ввиду вещественные признаки из исходного датасета, то это:  'carat', 'x'

## Попытка улучшить качество модели

Эта часть является творческой, и задания в ней немного сложнее, чем предыдущие. Поэтому за каждое из них можно получить до двух баллов.

**Задание 9* (2 балла)** Как можно заметить из анализа корреляционной матрицы в задании 3, между некоторыми признаками имеется сильная корреляция, что может быть индикатором проблемы *мультиколлинеарности*. Различия в порядке коэффициентов, выявленные в предыдущей задаче, также свидетельствуют об этом. Для решения этой проблемы можно либо исключить некоторые признаки из модели (например, если признак линейно зависим с какими-то другими, его можно исключить из модели, т.е. удалить из матрицы объект-признак и заново обучить модель).

Удалите из матриц Xtrain и Xtest признак, который наиболее сильно коррелирует с остальными. Заново обучите модель и оцените её качество. Улучшилось ли качество модели?
Попробуйте удалить какой-то другой признак (можете попробовать несколько вариантов). Помогло ли это улучшить качество модели?

In [326]:
# сильнее всех с другими признаками кореллирует признак 'x':
data.drop('price', axis = 1).corr().abs().sum().sort_values(ascending = False).head()

x            5.983354
carat        5.937227
z            5.933481
y            5.873453
cut_Ideal    4.000848
dtype: float64

In [327]:
# но при его удалении r2_score не улучшается:
# удаление других признаков и их комбинаций не привело к лучшим результатам
Xtrain_new = Xtrain.drop('x', axis=1)
Xtest_new = Xtest.drop('x', axis=1)
model.fit(Xtrain_new, y=ytrain)
ytest_predict = model.predict(Xtest_new)
ytrain_predict = model.predict(Xtrain_new)
print(f'r2-score на тренировочной выборке: {round(r2_score(ytrain, ytrain_predict),5)}')
print(f'r2-score на тестовой выборке: {round(r2_score(ytest, ytest_predict),5)}')

r2-score на тренировочной выборке: 0.92125
r2-score на тестовой выборке: 0.90246


**Задание 10* (2 балла)** Иногда генерация новых признаков помогает модели лучше находить взаимосвязи между целевой переменной и признаками. Попробуйте придумать новые признаки и добавить их в модель (можно черпать идеи из ноутбука занятия 3). Помогло ли это улучшить качество модели?

In [328]:
#добавил признак bad_brilliant - "плохой бриллиант", который равен 1, если у него худшие color, cut и clarity


In [329]:
def bad_brilliant(row):
    if (row.iloc[6:10].sum() == 0)  and row['color_J'] and (row.iloc[16:23].sum() == 0):
        return 1
    else:
        return 0

Xtrain['bad_brilliant']=Xtrain.apply(lambda x: bad_brilliant(x),axis=1)
Xtest['bad_brilliant']=Xtest.apply(lambda x: bad_brilliant(x),axis=1)

model.fit(Xtrain, y=ytrain)
ytest_predict = model.predict(Xtest)
ytrain_predict = model.predict(Xtrain)
print(f'r2-score на тренировочной выборке: {round(r2_score(ytrain, ytrain_predict),3)}')
print(f'r2-score на тестовой выборке: {round(r2_score(ytest, ytest_predict),3)}')

r2-score на тренировочной выборке: 0.922
r2-score на тестовой выборке: 0.914


In [330]:
#аналогично добавлял признак super_brilliant, отбирая бриллианты трёх лучших color, двух лучших cut, трёх лучших clarity

In [331]:
#добавление этих признаков не дало результата

## Выводы

Сделайте выводы, исходя из проделанной работы.

Какого наилучшего качества удалось добиться? Хорошее ли это качество на ваш взгляд? Что для этого вам пришлось сделать?

Также (по желанию) напишите, была ли эта домашняя работа для вас интересной.

### Выводы
- Удалось добиться качества r2_score = 91%
- Качество хорошее — но только для учебных целей, т.к. другие модели дают значительно лучший результа (см. Kaggle)
- Для достижения этого качества пришлось сделать:
    - первичную подготовку данных (na, лишние признаки,one-hot-coding)
    - стандартизацию признаков
    - процедуру деления выборок и обучения модели


Работа интересная. 
### Хочется узнать, 
- возможно ли достичь лучшего r2_score используя LinearRegression на этом датасете?